In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
%matplotlib notebook
from scipy.interpolate import griddata
from astropy.io import fits
import numpy as np
from scipy.spatial import cKDTree
from scipy.ndimage.filters import gaussian_filter

In [2]:
hdu_d = fits.open('CF4gp_new_64-z008_delta.fits')

In [3]:
t_d = hdu_d[0].data
np.shape(t_d)

(64, 64, 64)

In [4]:
t_d[63,10,10]

0.03911024

In [5]:
lania_in = pd.read_csv("laniakia_density.csv")
lania_in

,sgx,sgy,sgz,delta
0,-18.75,5.00,-60.0,-0.347969
1,-17.50,5.00,-60.0,-0.334302
2,-16.25,5.00,-60.0,-0.208867
3,-15.00,5.00,-60.0,-0.191881
4,-13.75,5.00,-60.0,-0.126498
...,...,...,...,...
1197005,-26.25,-51.25,150.0,0.599204
1197006,-25.00,-51.25,150.0,1.085904
1197007,-23.75,-51.25,150.0,0.721287
1197008,-22.50,-51.25,150.0,0.721287


In [6]:
ii_in = np.rint(np.round(32. + ((64./1000.)*lania_in["sgx"]),0)).astype(int)
jj_in = np.rint(np.round(32. + ((64./1000.)*lania_in["sgy"]),0)).astype(int)
kk_in = np.rint(np.round(32. + ((64./1000.)*lania_in["sgz"]),0)).astype(int)
delta = t_d[kk_in,jj_in,ii_in]

In [7]:
np.mean(delta),len(delta)

(0.0042303265, 1197010)

In [8]:
laniakia_density = pd.DataFrame(np.column_stack((lania_in.sgx,lania_in.sgy,lania_in.sgz,lania_in.delta,delta)),columns=['sgx','sgy','sgz','delta_2M++','delta_cf4'])
laniakia_density

,sgx,sgy,sgz,delta_2M++,delta_cf4
0,-18.75,5.00,-60.0,-0.347969,-0.108268
1,-17.50,5.00,-60.0,-0.334302,-0.108268
2,-16.25,5.00,-60.0,-0.208867,-0.108268
3,-15.00,5.00,-60.0,-0.191881,-0.108268
4,-13.75,5.00,-60.0,-0.126498,-0.108268
...,...,...,...,...,...
1197005,-26.25,-51.25,150.0,0.599204,0.066333
1197006,-25.00,-51.25,150.0,1.085904,0.066333
1197007,-23.75,-51.25,150.0,0.721287,0.066333
1197008,-22.50,-51.25,150.0,0.721287,0.065386


In [9]:
laniakia_density.to_csv('laniakia_density_2mpp_cf4.csv',index=False,header=True)

In [10]:
# Extract the coordinates and density fluctuation column as separate arrays
x = laniakia_density["sgx"]
y = laniakia_density["sgy"]
z = laniakia_density["sgz"]
density = delta

# Define the grid points to interpolate onto
xi, yi, zi = np.meshgrid(np.linspace(x.min(), x.max(), 256),
                         np.linspace(y.min(), y.max(), 256),
                         np.linspace(z.min(), z.max(), 256),
                         indexing='ij')

In [11]:
# Create a cKDTree object to search for the nearest neighbors
tree = cKDTree(np.column_stack((x, y, z)))

In [12]:
# Interpolate the density values onto the regular grid
distances, indices = tree.query(np.column_stack((xi.flatten(), yi.flatten(), zi.flatten())))
grid = np.reshape(density[indices], xi.shape)

In [15]:
np.save("reg_grid", grid)

In [13]:
# Define the radii to use for the spherical volumes
radii = np.arange(15., 200., 15)

r01=[];rms01=[]
# Calculate the variance of the smoothed field within each spherical volume
for r in radii:
    # Calculate the smoothing kernel size for this radius
    sigma = r / 2.35482  # Convert FWHM to sigma
    kernel_size = int(4 * sigma) + 1
    
    # Smooth the density field with a Gaussian filter
    smoothed = gaussian_filter(grid, sigma=sigma)
    
    # Calculate the variance within a spherical volume of radius r
    mask = np.sqrt(xi**2 + yi**2 + zi**2) <= r
    variance = np.var(smoothed[mask])
    
    # Print the result
    r01.append(r);rms01.append(np.sqrt(variance))
    print(f"Radius = {r:.1f} Mpc, RMS fluctuations = {np.sqrt(variance):.4f}")

Radius = 15.0 Mpc, RMS fluctuations = 0.2207
Radius = 30.0 Mpc, RMS fluctuations = 0.2677
Radius = 45.0 Mpc, RMS fluctuations = 0.2397
Radius = 60.0 Mpc, RMS fluctuations = 0.2153
Radius = 75.0 Mpc, RMS fluctuations = 0.1910
Radius = 90.0 Mpc, RMS fluctuations = 0.1738
Radius = 105.0 Mpc, RMS fluctuations = 0.1600
Radius = 120.0 Mpc, RMS fluctuations = 0.1453
Radius = 135.0 Mpc, RMS fluctuations = 0.1266
Radius = 150.0 Mpc, RMS fluctuations = 0.1081
Radius = 165.0 Mpc, RMS fluctuations = 0.0920
Radius = 180.0 Mpc, RMS fluctuations = 0.0785
Radius = 195.0 Mpc, RMS fluctuations = 0.0675


In [ ]:
import numpy as np
from scipy.ndimage import convolve

# Define the grid
size = 256
#grid = np.random.normal(size=(size, size, size))

# Define the radii to use for the spherical volumes
radii = np.arange(15., 200., 15)

rms_fluctuations = []
# Calculate the variance of the smoothed field within each spherical volume
for r in radii:
    # Define the coordinate arrays
    xi, yi, zi = np.indices((size, size, size)) - size // 2
    
    # Calculate the smoothing kernel size for this radius
    kernel_size = int(4 * r / 2.35482) + 1
    
    # Smooth the density field with a spherical tophat filter
    dist = np.sqrt(xi**2 + yi**2 + zi**2)
    smoothed = convolve(grid, (dist <= r).astype(float), mode='constant')
    smoothed /= (4 * np.pi / 3) * r**3  # Normalize by the volume
    
    # Calculate the variance within a spherical volume of radius r
    mask = dist <= r
    variance = np.var(smoothed[mask])
    
    # Print the result
    rms_fluctuations.append(np.sqrt(variance))
    print(f"Radius = {r:.1f} Mpc, RMS fluctuations = {np.sqrt(variance):.4f}")

In [ ]:
scale01 = np.array(r01) / 2.35482

In [ ]:
plt.scatter(np.array(r01) / 2.35482,rms01,color='black')
plt.grid(b=True, which='both', color='0.65',linestyle=':')
plt.tight_layout()
plt.show()

In [ ]:
def downsample(mygrid):
    """
    Downsample a 3D regular grid of density to half the resolution.

    Parameters:
    -----------
    grid : ndarray, shape (nx, ny, nz)
        The input grid of density.

    Returns:
    --------
    new_grid : ndarray, shape (nx/2, ny/2, nz/2)
        The downsampled grid of density.
    """

    # Get the size of the input grid
    nx, ny, nz = mygrid.shape

    # Create a new grid with half the size
    new_grid = np.zeros((nx//2, ny//2, nz//2))

    # Loop over every other cell in each dimension
    for i in range(0, nx, 2):
        for j in range(0, ny, 2):
            for k in range(0, nz, 2):
                # Add the neighbouring cells and take the average density
                new_grid[i//2, j//2, k//2] = np.mean(mygrid[i:i+2, j:j+2, k:k+2])

    return new_grid

In [ ]:
grids = [grid]
for i in range(10):
    print (i,np.std(grids[-1]))
    grids.append(downsample(grids[-1]))

In [ ]:
np.std(grid)

In [ ]:
grid2 = downsample(grid)

In [ ]:
np.std(grid2)

In [ ]:
grid3 = downsample(grid2)

In [ ]:
np.std(grid3)

In [ ]:
np.shape(grid)

In [ ]:
np.shape(grid2)

In [ ]:
np.shape(grid3)

In [ ]:
grid4 = downsample(grid3)

In [ ]:
np.std(grid4)

In [ ]:
np.shape(grid4)